In [1]:
import tensorflow as tf
import numpy as np
import importlib 
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *
import warnings
warnings.filterwarnings('ignore')

In [3]:
X_train = np.load('../../Project_Data/X_train_valid.npy')
y_train = np.load('../../Project_Data/y_train_valid.npy')
X_test = np.load('../../Project_Data/X_test.npy')
y_test = np.load('../../Project_Data/y_test.npy')

X_train_mean = X_train.mean(0)
X_train_var = np.sqrt(X_train.var(0))

X_train -= X_train_mean
X_train /= X_train_var
X_test -= X_train_mean
X_test /= X_train_var

X_train = np.transpose(X_train, (0, 2, 1))
X_test = np.transpose(X_test, (0, 2, 1))

y_train = y_train - np.min(y_train)
y_test = y_test - np.min(y_test)
    
print(y_train)

[2 3 0 ... 3 3 2]


In [4]:
def rnn_model_adam(features, labels, mode, params):
  
  
  num_hidden_layers = len(params['hidden_layers'])  
  
  outputs = features["x"]  

  model = params['model']
    
  activation = params.get("activation", tf.nn.tanh)

  print("Using activation - ", activation)

  for i in range(num_hidden_layers):  

    
      cell = model(name = 'cell'+str(i), num_units = params['hidden_layers'][i])
      outputs, state = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell, cell_bw = cell,
                                       inputs=outputs,
                                       dtype=tf.float64)
    
      #outputs = tf.layers.batch_normalization(outputs)
      outputs = outputs[1]
      outputs = activation(outputs)
      outputs = tf.nn.dropout(outputs, params['dropout'])

  print(type(outputs))
  print(outputs.shape)
  #FLatten the output of LSTM layers
  outputs = tf.contrib.layers.flatten(outputs) 

  print(type(outputs))
  print(outputs.shape)
  # FC Layer
  logits = tf.layers.dense(inputs=outputs, units=params['num_classes'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    
    


In [5]:
def rnn_model_rmsprop(features, labels, mode, params):
  
  
  num_hidden_layers = len(params['hidden_layers'])  
  
  outputs = features["x"]  

  model = params['model']
    
  activation = params.get("activation", tf.nn.tanh)

  print("Using activation - ", activation)

  for i in range(num_hidden_layers):  

    
      cell = model(name = 'cell'+str(i), num_units = params['hidden_layers'][i])
      outputs, state = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell, cell_bw = cell,
                                       inputs=outputs,
                                       dtype=tf.float64)
    
      #outputs = tf.layers.batch_normalization(outputs)
      outputs = outputs[1]
      outputs = activation(outputs)
      outputs = tf.nn.dropout(outputs, params['dropout'])

  print(type(outputs))
  print(outputs.shape)
  #FLatten the output of LSTM layers
  outputs = tf.contrib.layers.flatten(outputs) 

  print(type(outputs))
  print(outputs.shape)
  # FC Layer
  logits = tf.layers.dense(inputs=outputs, units=params['num_classes'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.RMSPropOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [6]:
def calc(eeg_classifier, X_train, y_train, X_test, y_test):

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        batch_size=50,
        num_epochs=20,
        shuffle=True)

    eeg_classifier.train(
        input_fn=train_input_fn,
        steps=100)

    eval_test_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)

    eval_train_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        num_epochs=1,
        shuffle=False)

    train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)
    test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
    print(train_results)
    print(test_results)

In [7]:
##BEST MODEL OBTAINED TILL NOW##

eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model_adam, model_dir="models/bidirectional_model1/",
                                        params = {'hidden_layers' : [64, 64, 64], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/bidirectional_model1/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5be0a5d710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, 

In [8]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model_rmsprop, model_dir="models/bidirectional_model2/",
                                        params = {'hidden_layers' : [64, 64, 64], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/bidirectional_model2/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5b9bafc160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x7f5bf29f8f28>
<class 'tensorflow.python.framework.ops.T

In [10]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model_rmsprop, model_dir="models/bidirectional_model3/",
                                        params = {'hidden_layers' : [64, 64], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/bidirectional_model3/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5be0a5d0b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x7f5bf29f8f28>
<class 'tensorflow.python.framework.ops.T

In [11]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model_adam, model_dir="models/bidirectional_model4/",
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/bidirectional_model4/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5b9b08da90>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x7f5bf29f8f28>
<class 'tensorflow.python.framework.ops.T

In [12]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model_adam, model_dir="models/bidirectional_model5/",
                                        params = {'hidden_layers' : [256, 256], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/bidirectional_model5/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5b9ba35278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x7f5bf29f8f28>
<class 'tensorflow.python.framework.ops.T

In [13]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model_adam, model_dir="models/bidirectional_model6/",
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.BasicRNNCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/bidirectional_model6/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5b8b009eb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x7f5bf29f8f28>
Instructions for updating:
This class is 

In [14]:
eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model_adam,
                                        params = {'hidden_layers' : [128, 128], 'num_classes' : 4, 
                                                  'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.GRUCell,
                                                 'dropout' : 0.5, 'activation' : tf.tanh})


calc(eeg_classifier, X_train, y_train, X_test, y_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpzjvggoej', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5bac667e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Using activation -  <function tanh at 0x7f5bf29f8f28>
Instructions for updating:
This class is equivalent a

In [18]:
losses = []
for e in tf.train.summary_iterator('models/bidirectional_model5/events.out.tfevents.1552822639.nndlproject'):
    for v in e.summary.value:
        if v.tag == 'loss':
            print(v.simple_value)
            losses.append(v.simple_value)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
1.3547964096069336
